# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
import requests
import json
import matplotlib.pyplot as plt
import pandas as pd
import citipy
import numpy as np
from config import api_key
from pprint import pprint

## Generate Cities List

In [12]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build partial query URL
query_url = f"{url}appid={api_key}&units={units}&q="
cities = ["Paris", "London", "Oslo", "Beijing"]

# set up lists to hold reponse info
long = []
lat = []
temp = []
humidity = []
clouds = []
wind = []
country = []


# Loop through the list of cities and perform a request for data on each
# api.openweathermap.org/data/2.5/find?q=London&units=imperial
for city in cities:
    
    response = requests.get(query_url + city).json()
    long.append(response['coord']['lon'])
    lat.append(response['coord']['lat'])
    temp.append(response['main']['temp'])
    humidity.append(response['main']['humidity'])
    clouds.append(response['clouds']['all'])
    wind.append(response['wind']['speed'])
    country.append(response['sys']['country'])

print(f"The latitude information received is: {lat}")
print(f"The longitude information received is: {long}")
print(f"The temperature information received is: {temp}")
print(f"The humidity information received is: {humidity}")
print(f"The cloud information received is: {clouds}")
print(f"The wind speed information received is: {wind}")
print(f"The country information received is: {country}")

# # Save config information
# url = "http://api.openweathermap.org/data/2.5/weather?"
# city = "Chicago"

# # Build query URL
# query_url = url + "appid=" + api_key + "&q=" + city

# # Get weather data
# weather_response = requests.get(query_url)
# weather_json = weather_response.json()

# # Get the temperature from the response
# print(f"The weather API responded with: {weather_json}.")
# pprint(weather_json)

The latitude information received is: [48.85, 51.51, 59.91, 39.91]
The longitude information received is: [2.35, -0.13, 10.75, 116.4]
The temperature information received is: [46.4, 47.23, 43.07, 47.16]
The humidity information received is: [93, 76, 84, 80]
The cloud information received is: [4, 4, 46, 0]
The wind speed information received is: [5.82, 3.36, 1.77, 2.24]
The country information received is: ['FR', 'GB', 'NO', 'CN']


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [3]:
#  Get the indices of cities that have humidity over 100%.


In [4]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression